In [1]:
import numpy as np
import scipy.integrate as integrate
import pandas as pd
import matplotlib.pyplot as plt
from financepy.models.black import *
from financepy.utils.global_types import *
import Fourier as fourier
import MJD as mjd

%load_ext autoreload
%autoreload 2

####################################################################
#  FINANCEPY BETA Version 0.370 - This build: 28 Oct 2024 at 20:26 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



In [2]:
def carr_madan_integrand(cf, logk, u, alpha):
    top = cf(complex(u, -(alpha+1.0))) * np.exp(complex(0.0, -u*logk))
    bottom = complex(alpha*alpha + alpha - u*u, (2.0*alpha + 1.0)*u)
    result = top/bottom
    return result.real 

def black_cf(forward, vol, t, u):
    lnf_factor = complex(0, u*np.log(forward))
    drift_and_diffusion_term = complex(-0.5*vol*vol*u*u, -0.5*vol*vol*u)
    return np.exp(lnf_factor + drift_and_diffusion_term * t)
  
def carr_madan_black_call_option(forward, vol, t, k, r, alpha = 1.5):
    cf = lambda u: black_cf(forward, vol, t, u)
    integrand = lambda u: carr_madan_integrand(cf, np.log(k), u, alpha)

    # lower limit is 0.0, upper limit is infinity
    result, error = integrate.quad(integrand, 0.0, np.inf)
    return np.exp(-r*t) * np.exp(-alpha*np.log(k)) / np.pi * result

### Different maturities

In [3]:
S0 = 100
T = 1
r = 0.05
vol = 0.5
q = 0.0
Ks = np.linspace(30, 170, 15)

df = pd.DataFrame()
df['Strike'] = Ks
  
DF = np.exp(-r*T)
forward = S0 * np.exp(r*T)

callOrPut = OptionTypes.EUROPEAN_CALL

my_price = [mjd.BS_Call_Option_Price(S0, K, vol, T, r, q) for K in Ks]
fourier_price = [carr_madan_black_call_option(forward, vol, T, K, r) for K in Ks]  

my_ivols = [mjd.ImpliedVolatility(call_value, K, T, S0, r, q) for call_value, K in zip(my_price, Ks)] 
fourier_ivols = [mjd.ImpliedVolatility(call_value,K,T,S0,r, q) for call_value, K in zip(fourier_price, Ks)] 

df['my price'] = np.array(my_price)
df['fourier'] = np.array(fourier_price)

df['my ivol'] = np.array(my_ivols)
df['fourier ivol'] = np.array(fourier_ivols)
           
df['my price % error'] = np.array(my_price)/np.array(fourier_price) - 1

df

,Strike,my price,fourier,my ivol,fourier ivol,my price % error
0,30.0,71.514004,71.514004,0.5,0.5,0.000000e+00
1,40.0,62.256431,62.256431,0.5,0.5,2.220446e-16
2,50.0,53.457881,53.457881,0.5,0.5,-1.110223e-16
3,60.0,45.358636,45.358636,0.5,0.5,-7.771561e-16
4,70.0,38.116228,38.116228,0.5,0.5,8.881784e-16
5,80.0,31.792518,31.792518,0.5,0.5,-6.661338e-16
6,90.0,26.372715,26.372715,0.5,0.5,-2.220446e-16
7,100.0,21.792604,21.792604,0.5,0.5,4.440892e-16
8,110.0,17.962324,17.962324,0.5,0.5,8.881784e-16
9,120.0,14.783345,14.783345,0.5,0.5,-1.110223e-16
